<a href="https://colab.research.google.com/github/edwin-19/nemo_example/blob/master/ASR_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nemo_toolkit['all']
!pip install datasets
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.1/151.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 8.6 MB/s eta 0:00:

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.4 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset, Audio
import librosa
import soundfile as sf
from matplotlib import pyplot as plt
import nemo
import nemo.collections.asr as nemo_asr

from transformers import AutoProcessor, Wav2Vec2ForCTC
import jiwer
import torch
import evaluate
from tqdm import tqdm
from IPython.display import Audio

# Load components
- Load evaluate
- Load model
- Load random dataset

In [3]:
wer = evaluate.load("wer")
cer = evaluate.load("cer")

In [19]:
dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
# Conformer
asr_model = nemo_asr.models.EncDecCTCModelBPE.from_pretrained("nvidia/stt_en_conformer_ctc_small")

# Fast Conformer
fast_asr_model = nemo_asr.models.EncDecRNNTBPEModel.from_pretrained(model_name="nvidia/stt_en_fastconformer_transducer_large")

[NeMo I 2023-10-03 16:33:17 mixins:170] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2023-10-03 16:33:18 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/NeMo_ASR_SET/English/v2.0/train/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 64
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/NeMo_ASR_SET/English/v2.0/train/audio__OP_0..4095_CL_.tar
    
[NeMo W 2023-10-03 16:33:18 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /data/ASR/LibriSpeech/librispeech_withs

[NeMo I 2023-10-03 16:33:18 features:289] PADDING: 0
[NeMo I 2023-10-03 16:33:20 save_restore_connector:249] Model EncDecCTCModelBPE was successfully restored from /root/.cache/huggingface/hub/models--nvidia--stt_en_conformer_ctc_small/snapshots/f879b51de584983383de815ce87d25469b2abbf3/stt_en_conformer_ctc_small.nemo.


[NeMo I 2023-10-03 16:33:37 mixins:170] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2023-10-03 16:33:39 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 1
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20
    min_duration: 0.1
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: fully_randomized
    bucketing_batch_size: null
    
[NeMo W 2023-10-03 16:33:39 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    num_workers: 8
    pin_m

[NeMo I 2023-10-03 16:33:39 features:289] PADDING: 0


[NeMo W 2023-10-03 16:33:40 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
      warnings.warn("dropout option adds dropout after all but last "
    


[NeMo I 2023-10-03 16:33:41 rnnt_models:211] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2023-10-03 16:33:42 save_restore_connector:249] Model EncDecRNNTBPEModel was successfully restored from /root/.cache/huggingface/hub/models--nvidia--stt_en_fastconformer_transducer_large/snapshots/63d5e0b706c82b5cc14ba0d9955393af7e57b863/stt_en_fastconformer_transducer_large.nemo.


In [7]:
dataset

Dataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
    num_rows: 73
})

In [20]:
pred_text = asr_model.transcribe(dataset['file'], batch_size=1)

Transcribing:   0%|          | 0/73 [00:00<?, ?it/s]

In [21]:
fast_pred_text = fast_asr_model.transcribe(dataset['file'], batch_size=1)

Transcribing:   0%|          | 0/73 [00:00<?, ?it/s]

In [22]:
dataset['text'][:10], pred_text[:10]

(['MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL',
  "NOR IS MISTER QUILTER'S MANNER LESS INTERESTING THAN HIS MATTER",
  'HE TELLS US THAT AT THIS FESTIVE SEASON OF THE YEAR WITH CHRISTMAS AND ROAST BEEF LOOMING BEFORE US SIMILES DRAWN FROM EATING AND ITS RESULTS OCCUR MOST READILY TO THE MIND',
  "HE HAS GRAVE DOUBTS WHETHER SIR FREDERICK LEIGHTON'S WORK IS REALLY GREEK AFTER ALL AND CAN DISCOVER IN IT BUT LITTLE OF ROCKY ITHACA",
  "LINNELL'S PICTURES ARE A SORT OF UP GUARDS AND AT EM PAINTINGS AND MASON'S EXQUISITE IDYLLS ARE AS NATIONAL AS A JINGO POEM MISTER BIRKET FOSTER'S LANDSCAPES SMILE AT ONE MUCH IN THE SAME WAY THAT MISTER CARKER USED TO FLASH HIS TEETH AND MISTER JOHN COLLIER GIVES HIS SITTER A CHEERFUL SLAP ON THE BACK BEFORE HE SAYS LIKE A SHAMPOOER IN A TURKISH BATH NEXT MAN",
  'IT IS OBVIOUSLY UNNECESSARY FOR US TO POINT OUT HOW LUMINOUS THESE CRITICISMS ARE HOW DELICATE IN EXPRESSION',
  'ON THE GENERAL PRINCIPLES OF ART M

In [16]:
asr_model.export('model.onnx')
fast_asr_model.export('fast_conformer_model.onnx')

[NeMo W 2023-10-03 16:28:22 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:2033: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input audio_signal
      warnings.warn(
    
[NeMo W 2023-10-03 16:28:22 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:2033: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input length
      warnings.warn(
    
[NeMo W 2023-10-03 16:28:22 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:2033: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input logprobs
      warnings.warn(
    
[NeMo W 2023-10-03 16:28:22 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/nemo/coll

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

[NeMo I 2023-10-03 16:28:48 exportable:130] Successfully exported EncDecCTCModelBPE to model.onnx


[NeMo W 2023-10-03 16:28:49 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:2033: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input outputs
      warnings.warn(
    
[NeMo W 2023-10-03 16:28:49 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:2033: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input encoded_lengths
      warnings.warn(
    


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

[NeMo I 2023-10-03 16:29:20 exportable:130] Successfully exported ConformerEncoder to encoder-fast_conformer_model.onnx


[NeMo W 2023-10-03 16:29:21 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:2033: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input encoder_outputs
      warnings.warn(
    
[NeMo W 2023-10-03 16:29:21 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:2033: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input targets
      warnings.warn(
    
[NeMo W 2023-10-03 16:29:21 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:2033: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input target_length
      warnings.warn(
    
[NeMo W 2023-10-03 16:29:21 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

[NeMo I 2023-10-03 16:29:22 exportable:130] Successfully exported RNNTDecoderJoint to decoder_joint-fast_conformer_model.onnx


(['encoder-fast_conformer_model.onnx',
  'decoder_joint-fast_conformer_model.onnx'],
 ['nemo.collections.asr.modules.conformer_encoder.ConformerEncoder exported to ExportFormat.ONNX',
  'nemo.collections.asr.modules.rnnt.RNNTDecoderJoint exported to ExportFormat.ONNX'])

In [28]:
# len(fast_pred_text[0])
wer.compute(predictions=fast_pred_text[0], references=dataset['text']), cer.compute(predictions=fast_pred_text[0], references=dataset['text'])

(1.0026086956521738, 0.8268183274591477)

In [24]:
wer.compute(predictions=pred_text, references=dataset['text']), cer.compute(predictions=pred_text, references=dataset['text'])

(1.0026086956521738, 0.8297020185837872)